# Paint the circle Project

This project identifies and paints circles given in RGB images. Specifically, each input image contains a rectangle, a triangle and a circle. The image is fed into a neural netork, whose outputs are the center and radius of the circle within the image. The domain considered is "[0,1]x[0,1] "box". PyTorch is the platform used. The goal is to paint the circle.

### **DataSet fils:**
  Three directories named "train", "validation", and "test" each containing the following:
  1. a directory called "images" with RGB png files of size 128x128 (so image that is read has shape (3,128,128))
  2. a file "labels.txt" containing the center points (as x,y) and radius of the circles for all images in "images" directory
 
 
### **General structure:**
1. Dataset class definition.
2. Creation of Data Loaders.
3. Neural network definition.
4. Loss function definition.
5. Total number of model's parameters calculation.
6. Model creation and choice of optimizer.
7. Total number of op's per forward feed calc class definition.
8. Total number of op's per forward feed calculation. 
9. Images, target and net labels viewer.
10. Model validation - returns avg loss per image for a model and loader
11. Definition train model function for training the network.
12. Activation of the train model function.
13. Plot for visualisation of train and validation losses from the training process.
14. Definitions of Save/Load functions for the Model.
15. Example use of saving and loading functions.
15. Visualizing images - Painting circles prodiced by network on images from a given loader.
16. Example of how to paint circles produced by model.
 
 
### ** Useful links: **
1. PyTorch master tutorial - VERY useful: https://pytorch.org/docs/master/nn.html
2. PyTorch optimizers: https://pytorch.org/docs/master/optim.html
3. A list of possible reasons why things go wrong: https://blog.slavv.com/37-reasons-why-your-neural-network-is-not-working-4020854bd607#74de

In [ ]:
import numpy as np
import PIL.Image as Image
from PIL import ImageDraw

import torch.nn as nn

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import glob
import datetime

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Dataset class definition

In [ ]:
class ShapesDataset(Dataset):
    
    def __init__(self, dataset_dir):
        """
        Initializing dataset by generating a dicitonary of labels, where an image file name is the key 
        and its labels are the contents of that entry in the dictionary. Images are not loaded. This way it
        is possible to iterate over arbitrarily large datasets (limited by labels dicitonary fitting 
        in memory, which is not a problem in practice)
        
        Args:
            dataset_dir : path to directory with images and labels. In this directory we expect to find
                          a directory called "images" containing the input images, and a file called 
                          "labels.txt" containing desired labels (coefficients)
        """
        
        self.dataset_dir = dataset_dir
        self.labels_dict = self.gen_labels_dict()
        self.images_keys = list(self.labels_dict)  # getting the keys of the dictionary as list
        self.images_keys.sort()                    # sorting so as to have in alphabetical order 

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, index):    
        """
        This funtion makes it possible to iterate over the ShapesDataset
        Args:
            index: running index of images
            
        Returns:
            sample: a dicitionary with three entries:
                    1. 'image'  contains the image
                    2. 'labels' contains labels (coeffs) corresponding to image
                    3. 'fname'  contains name of file (image_key) - may be useful for debugging
        """
        image_key = self.images_keys[index]     # recall - key is the file name of the corresponding image
        image = np.array(Image.open(image_key)) # image has shape: (128, 128, 3)
        image = image/255.0                     # simple normalization - just to maintain small numbers
        image = np.transpose(image, (2, 0, 1))  # network needs RGB channels to be first index
        labels = self.labels_dict[image_key]
        sample = {'image': image, 'labels': labels, 'fname':image_key}
        
        return sample
    
    
    def gen_labels_dict(self):
        """
        This fucntion generates a dictionary of labels
        
        Returns:
            labels_dict: the key is image file name and the value is the corresponding 
            array of labels  
        """
        
        labels_fname = self.dataset_dir + "/labels.txt"
        labels_dict = {}
        with open(labels_fname, "r") as inp:
            for line in inp:
                line = line.split('\n')[0]                                      # remove '\n' from end of line 
                line = line.split(',')
                key  = self.dataset_dir + '/images/' + line[0].strip() + ".png" # image file name is the key
                del line[0]
                
                list_from_line = [float(item) for item in line]
                labels_dict[key] = np.asarray(list_from_line, dtype=np.float32)
                        
        return labels_dict             


# Creation of Data Loaders

In [ ]:
train_dir      = "./train/"
validation_dir = "./validation/"
test_dir       = "./test/"


train_dataset = ShapesDataset(train_dir)

train_loader = DataLoader(train_dataset, 
                          batch_size=32,
                          shuffle=True)

validation_dataset = ShapesDataset(validation_dir)

validation_loader = DataLoader(validation_dataset, 
                               batch_size=32,
                               shuffle=False)

test_dataset = ShapesDataset(test_dir)

test_loader = DataLoader(test_dataset, 
                          batch_size=1,
                          shuffle=False)


print("train loader examples     :", len(train_dataset)) 
print("validation loader examples:", len(validation_dataset))
print("test loader examples      :", len(test_dataset))

# Neural network definition

In [ ]:
class CircleNet(nn.Module):    # nn.Module is parent class  
    def __init__(self):
        super(CircleNet, self).__init__()  #calls init of parent class

        self.conv1 = nn.Conv2d(3, 10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv3 = nn.Conv2d(20, 50, kernel_size=3)
        self.conv4 = nn.Conv2d(50, 100, kernel_size=3)
        self.conv_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(3600, 2400)
        self.fc2 = nn.Linear(2400, 1200)
        self.fc3 = nn.Linear(1200, 3)
        
    def forward(self, x):
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """
        batch_size = x.shape[0]
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv_drop(self.conv2(x)), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv_drop(self.conv3(x)), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv_drop(self.conv4(x)), 2))
        #print(x.shape)
        
        x = x.view(batch_size, -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, training=self.training)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.dropout(x, training=self.training)
        
        x = self.fc3(x)
        out = x        
        return out                   

# Loss function definition

In [ ]:
def my_loss(outputs, labels):
    
    """
    Args:
        outputs - output of network ([batch size, 3]) 
        labels  - desired labels  ([batch size, 3])
    """
    
    loss = torch.zeros(1, dtype=torch.float, requires_grad=True)
    loss = loss.to(device)

    loss_calc = outputs.add(-labels)
    loss_calc = loss_calc.pow(2)
    loss_calc = loss_calc.sum(dim=1)
    loss_calc = loss_calc.sqrt()
    loss = loss_calc.mean()
                                        
    return loss

# Total number of model's parameters calculation

In [ ]:
def get_train_params_num(model):
    """
    This fucntion returns the number of trainable parameters of neural network model
    You may want to call it after you create your model to see how many parameteres the model has
    Args:
        model - neural net to examine
    """
    
    #filter given iterable with a function that tests each element in the iterable to be true or not
    model_parameters = filter(lambda p: p.requires_grad == True, model.parameters()) 
    params_num = sum([np.prod(p.size()) for p in model_parameters])
    return params_num


# Model creation and choice of optimizer

In [ ]:
model = CircleNet().to(device)
print ("Number of model trainable parameters:", get_train_params_num(model))

#----------------------------------------------
#  Choice of optimizer:
#----------------------------------------------
optimizer = optim.Adam(model.parameters(), lr=0.00005)

# Total number of op's per forward feed calc class definition

In [ ]:
def calc_ops(inp_size, net_struct):
    """
    Calculates a rough number of operations for a given network topology
    Args:
        inp_size - (W,H) of input 
        net_struct - list of tuples describing structure of network. 
        
        Example:
         (
         
          ('conv2d', (3, 8, 3, 1, 0)),  # cin, cout, kernel, stride, pad
          ('conv2d': 83, 8, 3, 1, 0)),
          ('MaxPool2d', (2,2)),         # kernel, stride
          ('fc': (64, 8)),      
          ('fc': (8, 4))
          
          )
         
    """
    
    ops = 0
    W, H = inp_size
    for curr_item in net_struct:
        if curr_item[0] == 'conv2d':
            cin = curr_item[1][0]
            cout = curr_item[1][1]
            kernel = curr_item[1][2]
            stride = curr_item[1][3]
            pad = curr_item[1][4]
            W = (W +2*pad - kernel)/stride + 1
            H = (H +2*pad - kernel)/stride + 1
            curr_ops = (W*H*cin*cout*kernel*kernel)/stride
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
        elif curr_item[0] == 'MaxPool2d':
            kernel = curr_item[1][0]
            stride = curr_item[1][1]
            W = (W - kernel)/stride + 1
            H = (H - kernel)/stride + 1
        else:
            curr_ops = curr_item[1][0] * curr_item[1][1]
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
            
    return int(ops)

# Total number of op's per forward feed calculation

In [ ]:
inp_size = (128,128)

# place network ropology in example_net below to obtain an estimated number of operations for the network
example_net = (('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('fc', (2883, 4)))

ops = calc_ops(inp_size, example_net)
print()
print("Total ops: {:,}".format(ops))

# Images, target and net labels viewer

In [ ]:
"""
View first image of a given number of batches assuming that model has been created. 
Currently, lines assuming model has been creatd, are commented out. Without a model, 
you can view target labels and the corresponding images.
This is given to you so that you may see how loaders and model can be used. 
"""

loader = train_loader # choose from which loader to show images
bacthes_to_show = 2
with torch.no_grad():
    for i, data in enumerate(loader, 0):      # 0 means that counting starts at zero
        inputs = (data['image']).to(device)   # has shape (batch_size, 3, 128, 128)
        labels = (data['labels']).to(device)  # has shape (batch_size, 3)
        img_fnames = data['fname']            # list of length batch_size
        
        outputs = model(inputs.float())
        img = Image.open(img_fnames[0])
        
        print ("showing image: ", img_fnames[0])
        
        labels_str = [ float(("{0:.2f}".format(x))) for x in labels[0]]  #labels_np_arr]
        
        outputs_np_arr = outputs[0] # using ".numpy()" to convert tensor to numpy array
        outputs_str = [ float(("{0:.2f}".format(x))) for x in outputs_np_arr]
        print("Target labels :", labels_str)
        print("network coeffs:", outputs_str)
        print()
        img.show()
        
        if (i+1) == bacthes_to_show:
            break
        

# Model validation - returns avg loss per image for a model and loader

In [ ]:
def validate_model(model, loader):
    """
    This function parses a given loader and returns the avergae (per image) loss 
    (as defined by "my_loss") of the entire dataset associated with the given loader.
    
    Args:
        model  - neural network to examine
        loader - where input data comes from (train, validation, or test)
        
    returns:
        average loss per image in variable named "avg_loss"
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    with torch.no_grad():        
        val_loss = 0
        avg_loss = 0
        iters = 0
        for batch_idx, data_dict in enumerate(loader):
            val_data = (data_dict['image']).to(device)
            val_labels = (data_dict['labels']).to(device)
            validated_data = model(val_data.float())
            val_loss = my_loss(validated_data, val_labels)  
            avg_loss = avg_loss + val_loss
            iters += 1
    avg_loss = (avg_loss / iters)
    
    model.train()  #back to default
    
    return avg_loss

# Definition train model function for training the network

In [ ]:
def train_model(model,
                optimizer,
                train_loader,
                validation_loader,
                train_losses,
                validation_losses,
                epochs=1):
    
    """
    Trains a neural network. 
    Args:
        model               - model to be trained
        optimizer           - optimizer used for training
        train_loader        - loader from which data for training comes 
        validation_loader   - loader from which data for validation comes (maybe at the end, you use test_loader)
        train_losses        - adding train loss value to this list for future analysis
        validation_losses   - adding validation loss value to this list for future analysis
        epochs              - number of runs over the entire data set 
    """
    
    model.train()
   
    for ep in range(epochs):
        total_loss = 0
        iters = 0
        for batch_idx, data_dict in enumerate(train_loader):                  
            data = (data_dict['image']).to(device)
            labels = (data_dict['labels']).to(device)
            optimizer.zero_grad()
            trained_data = model(data.float())            
            loss = my_loss(trained_data, labels)
            loss.backward()
            optimizer.step()
            total_loss = total_loss + loss
            
            # logging
            # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              #      ep, batch_idx *  train_loader.batch_size, len(train_loader.dataset),
              #      100. * batch_idx / len(train_loader), loss.item()))
            iters += 1
        
        #add the avg loss of current eopch
        avg_loss = total_loss/iters
        train_losses.append(avg_loss)
        
        #validate
        valid_loss = validate_model(model, validation_loader)
        print("Final epoch: Train epoch: {}, validation_loss: {}, train_loss: {}".format(ep, valid_loss, avg_loss))
        validation_losses.append(valid_loss)
        
    return 

# Activation of the train model function

In [ ]:
# Using two lists (train_losses, validation_losses) containing history of losses 
# (i.e., loss for each training epoch) for train and validation sets. 
# If thess are not defined, we define them. Otherwise, the function train_model
# updates these two lists (by adding loss values when it is called for further training) 
# in order to be able to visualize train and validation losses

if not 'train_losses' in vars():
    train_losses = []
if not 'validation_losses' in vars():
    validation_losses = []


train_model(model, 
            optimizer,
            train_loader, 
            validation_loader, 
            train_losses, 
            validation_losses,
            epochs=500)

# Plot for visualisation of train and validation losses from the training process

In [ ]:
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

iteration = np.arange(0., len(train_losses))
plt.plot(iteration, train_losses, 'g-', iteration, validation_losses, 'r-')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

# Definitions of Save/Load functions for the Model

In [ ]:
def save(model, train_losses, validation_losses, save_dir):
    """
    saving model, train losses, and validation losses
    Args:
        model              - NN to be saved
        train_losses       - history of losses for training dataset
        validation_losses  - history of losses for validation dataset
        save_dir           - directory where to save the above
    """
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    files = glob.glob(save_dir + '*')
    for f in files:
        os.remove(f) 
        
    torch.save(model, save_dir + "/model.dat")
    
    train_losses_f = open(save_dir + "/train_losses.txt", "wt")
    train_losses_f.writelines( "%.3f\n" % item for item in train_losses)
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "wt")
    validation_losses_f.writelines( "%.3f\n" % item for item in validation_losses)

    return
   
def load(save_dir):
    """
    loading model, train losses, and validation losses
    Args:
       save_dir  - dir name from where to load 
    """
    
    model = torch.load(save_dir + "/model.dat") 
    
    train_losses_f = open(save_dir + "/train_losses.txt", "rt")
    train_losses   = train_losses_f.readlines()
    train_losses   = [float(num) for num in train_losses]
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "rt")
    validation_losses   = validation_losses_f.readlines()
    validation_losses   = [float(num) for num in validation_losses]
    
    return (model, train_losses, validation_losses)
   

# Example use of saving and loading functions

In [ ]:
# Create a directory, for example "./saves_12/", where you place your saved models

save(model, train_losses, validation_losses, "./saves/")

model, train_losses, validation_losses = load("./saves/")

# Visualizing images - Painting circles prodiced by network on images from a given loader

In [ ]:
def paint_loader_circles(model, loader, out_dir):
    """
    This fucntion receives a model, a loader and an output directory. 
    For each image in the loader it paints a circle that the model identifies. 
    The images are saved in the given out_dir diretory. 
    Args:
        model   - network for idneitfying circles
        loader  - input data to use 
        out_dir - ouptut directory name (e.g.: 'draws/'). If directory does not exist, it is created.
                  If it exists, its files are deleted.
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    k = 0
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files = glob.glob(out_dir + '*')
    for f in files:
        os.remove(f) 
          
    for data in loader:
        # get inputs
        inputs = (data['image']).to(device)
        labels = (data['labels']).to(device)  # not using 
        img_fnames = data['fname'] 
            
        # forward
        outputs = model(inputs.float())
        curr_batch_size = np.shape(outputs)[0]
        image_size = np.shape(inputs[0])  # image_size = [3, w, h]
        _, width, height = image_size
        assert (width == height)
        
        for i in range (curr_batch_size): 
            x0 = (outputs[i, 0].item()) * width
            y0 = (1-outputs[i, 1].item()) * height
            r  = outputs[i, 2].item() * width #assume width=height here. Otherwise, circle becomes ellipse
   
            fname = img_fnames[i]
            k+=1
            print (str(k) + ".   " + fname)

            img = Image.open(fname)
            draw = ImageDraw.Draw(img, 'RGBA')
    
            draw.ellipse((x0 - r, y0 - r, x0 + r ,y0 + r), fill=(160, 64, 0, 90), outline=None)
    
            img.save(out_dir + fname.split('/')[-1])
    
        
    model.train()  #back to default
    return

# Example of how to paint circles produced by model

In [ ]:
# Painting circles on images from validation loader and placing them in directory "./validation/draw/". 
# Notice that if the painted circle is seen only partly, it means that it is not inside the 
# [0,1]x[0,1] "box", which is the domain considered.


#paint_loader_circles(model, validation_loader, './validation/draw/')
paint_loader_circles(model, test_loader, './test/draw/')